# Experiments for image encoding evaluation

## Setup

### Google Drive setup

In [ ]:
import os
import subprocess
from pathlib import Path

GEQIE_PATH = Path(os.getcwd()).parent
WORK_DIR = Path(".")  # directory for logs and outputs, overridden in Colab scenario

DRIVE_MOUNT_PATH = Path("/content/drive")
_DRIVE_WORK_DIR = DRIVE_MOUNT_PATH / "MyDrive" / "Colab Notebooks" / "GEQIE"  # overrides WORK_DIR in Colab

try:
    from google.colab import drive

    print("Environment is Google Colab.")
    WORK_DIR = _DRIVE_WORK_DIR

    GEQIE_PATH = Path("/content/geqie")
    print(f"GEQIE path set to '{GEQIE_PATH}'")

    print(f"Mounting Google Drive '{DRIVE_MOUNT_PATH}'...")
    drive.mount(DRIVE_MOUNT_PATH.as_posix())

    print("Installing dependencies...")

    ! git clone https://github.com/merQlab/geqie.git $GEQIE_PATH 2>/dev/null
    ! uv pip install $GEQIE_PATH 2>/dev/null
    ! uv pip install -r $GEQIE_PATH/experiments/requirements/requirements.in 2>/dev/null

    import subprocess
    result = subprocess.run(['nvidia-smi'])
    if result.returncode == 0:
        print("NVIDIA GPU detected. Installing qiskit-aer-gpu")
        ! uv pip install qiskit-aer-gpu 2>/dev/null
    else:
        print("No NVIDIA GPU detected.")

    print("Dependencies installed successfully.")
    os.kill(os.getpid(), 9)
    print("Restarting runtime...")
except ModuleNotFoundError as e:
    print("Environment is not Google Colab... skipping.")
except Exception as e:
    print("Error encountered: " + str(e))

Environment is not Google Colab... skipping.


### Imports

In [2]:
from itertools import product
from dataclasses import dataclass
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image, ImageOps, ImageDraw
from scipy.stats import pearsonr
from tqdm import tqdm

from qiskit import transpile
from qiskit_aer.noise import NoiseModel
from qiskit_aer.noise.errors import depolarizing_error

In [3]:
import geqie
from geqie.encodings import frqi, ifrqi, mcqi, ncqi, neqr, qrci, qualpi

### Constants

In [4]:
CURRENT_PATH = Path(os.path.abspath("."))

GRAYSCALE_METHODS = [
    frqi,
    neqr,
]
GRAYSCALE_PATH = CURRENT_PATH.parent / "experiments" / "generated_images" / "grayscale"
GRAYSCALE_IMAGE_PATHS = sorted(GRAYSCALE_PATH.glob("*.png"))

RGB_METHODS = [
    ifrqi,
    mcqi,
]
RGB_PATH = CURRENT_PATH.parent / "experiments" / "generated_images" / "rgb"
RGB_IMAGE_PATHS = sorted(RGB_PATH.glob("*.png"))

### Helpers

In [5]:
def plot_images_side_by_side(original, retrieved):
    image_original = Image.fromarray(original)
    draw_original = ImageDraw.Draw(image_original)
    image_retrieved = Image.fromarray(retrieved)
    draw_retrieved = ImageDraw.Draw(image_retrieved)
    display(image_original.resize((image_original.width*30, image_original.height*30), resample=Image.NEAREST))
    display(image_retrieved.resize((image_retrieved.width*30, image_retrieved.height*30), resample=Image.NEAREST))

In [6]:
def PSNR(original, retrieved):
    mse = np.mean((original - retrieved) ** 2)
    if mse == 0:
        return float("inf")
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

## Experiments

### Configuration

In [7]:
@dataclass
class ResultsRow:
    method_name: str
    image_name: str
    pcc_clean: float
    psnr_clean: float
    pcc_noised: float
    psnr_noised: float

    circuit_depth: int
    circuit_size: int
    cnot_count: int
    n_qubits: int
    n_shots: int
    noise_model: str

In [8]:
N_SHOTS = 1024

CSV_RESULTS_PATH = WORK_DIR / "results" 
CSV_RESULTS_PATH.mkdir(parents=True, exist_ok=True)

RESULTS_CSV_COLUMNS = ResultsRow.__annotations__.keys()
RESULTS_CSV_FILENAME_TEMPLATE = "{time_of_execution}_{colormode}.csv"

#### Transpilation

In [9]:
TRANSPILATION_BASIS = ["cx", "u"]

def get_cnot_count(circuit):
    transpiled_circuit = transpile(circuit, basis_gates=TRANSPILATION_BASIS, optimization_level=0)
    return transpiled_circuit.count_ops().get("cx", 0)

### Grayscale experiment

In [10]:
time_of_execution = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
GRAYSCALE_RESULTS_PATH = CSV_RESULTS_PATH / RESULTS_CSV_FILENAME_TEMPLATE.format(
    time_of_execution=time_of_execution,
    colormode="grayscale",
)

df_results_grayscale = pd.DataFrame(columns=RESULTS_CSV_COLUMNS)


for (method, image_path) in tqdm(list(product(GRAYSCALE_METHODS[:1], GRAYSCALE_IMAGE_PATHS))):
    method_name = method.__name__.split('.')[-1]
    image_name = image_path.stem

    image = Image.open(image_path)
    image = ImageOps.grayscale(image)
    image = np.asarray(image)

    circuit = geqie.encode(method.init_function, method.data_function, method.map_function, image)

    # Create a global noise model with depolarizing error
    noise_model = NoiseModel()
    depolarizing_error_probability = 0.25
    depol_error = depolarizing_error(depolarizing_error_probability, circuit.num_qubits)
    noise_model.add_all_qubit_quantum_error(depol_error, ["unitary"])

    result = geqie.simulate(circuit, N_SHOTS)
    retrieved_image = method.retrieve_function(result)

    result_noised = geqie.simulate(circuit, N_SHOTS, noise_model=noise_model)
    retrieved_noised_image = frqi.retrieve_function(result_noised)

    pcc_clean_image, _ = pearsonr(image.flatten(), retrieved_image.flatten())
    pcc_noised_image, _ = pearsonr(image.flatten(), retrieved_noised_image.flatten())
    psnr_clean_image = PSNR(image, retrieved_image)
    psnr_noised_image = PSNR(image, retrieved_noised_image)
    
    results = ResultsRow(
        method_name=method_name,
        image_name=image_name,
        pcc_clean=pcc_clean_image,
        psnr_clean=psnr_clean_image,
        pcc_noised=pcc_noised_image,
        psnr_noised=psnr_noised_image,

        cnot_count=get_cnot_count(circuit),
        circuit_size=circuit.size(),
        circuit_depth=circuit.depth(),
        n_qubits=circuit.num_qubits,
        n_shots=N_SHOTS,
        noise_model=f"depolarizing_{depolarizing_error_probability*100:.0f}pct",
    )

    df_results_grayscale.loc[len(df_results_grayscale)] = results.__dict__
    
df_results_grayscale.to_csv(GRAYSCALE_RESULTS_PATH, index=False)
print(f"Results saved to '{GRAYSCALE_RESULTS_PATH}'")

100%|██████████| 24/24 [02:29<00:00,  6.22s/it]

Results saved to 'results\2025_12_09__20_28_09_grayscale.csv'
